In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

In [2]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
users = pd.read_csv('users.csv')

In [3]:
data = pd.merge(ratings, movies, on='MovieID')
data = pd.merge(data, users, on='UserID')
data.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [4]:
movie_id_to_name = pd.Series(movies.Title.values, index=movies.MovieID).to_dict()


In [5]:
user_counts = data['UserID'].value_counts()
filtered_users = user_counts[user_counts >= 5].index
data = data[data['UserID'].isin(filtered_users)]
print(f"Data after filtering: {data.shape}")

Data after filtering: (1000209, 10)


In [6]:
movie_counts = data['MovieID'].value_counts()
filtered_movies = movie_counts[movie_counts >= 5].index
data = data[data['MovieID'].isin(filtered_movies)]
print(f"Data after filtering: {data.shape}")

Data after filtering: (999611, 10)


In [7]:
data['Timestamp'] = pd.to_datetime(data['Timestamp'], unit='s')
data.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code
0,1,1193,5,2000-12-31 22:12:40,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,2000-12-31 22:35:09,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,2000-12-31 22:32:48,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,2000-12-31 22:04:35,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,2001-01-06 23:38:11,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067


In [8]:
user_encoder = LabelEncoder()
data["EncodedUserID"] = user_encoder.fit_transform(data["UserID"]) + 1

In [9]:
movie_encoder = LabelEncoder()
data["EncodedMovieID"] = movie_encoder.fit_transform(data["MovieID"]) + 1

In [10]:
print(data['EncodedMovieID'].nunique())  
print(data['EncodedMovieID'].max()) 

3416
3416


In [11]:
movie_id_to_name = pd.Series(movies.Title.values, index=movies.MovieID).to_dict()
encoded_movie_id_to_name = {
    movie_encoder.transform([k])[0] + 1: v for k, v in movie_id_to_name.items() if k in movie_encoder.classes_
}
encoded_movie_name_to_id = {v: k for k, v in encoded_movie_id_to_name.items()}

In [12]:
data = data.sort_values(["EncodedUserID", "Timestamp"]).reset_index(drop=True)
print(data[['EncodedUserID','Timestamp']].head())

   EncodedUserID           Timestamp
0              1 2000-12-31 22:00:19
1              1 2000-12-31 22:00:55
2              1 2000-12-31 22:00:55
3              1 2000-12-31 22:00:55
4              1 2000-12-31 22:01:43


In [13]:
data.head()

,UserID,MovieID,Rating,Timestamp,Title,Genres,Gender,Age,Occupation,Zip-code,EncodedUserID,EncodedMovieID
0,1,3186,4,2000-12-31 22:00:19,"Girl, Interrupted (1999)",Drama,F,1,10,48067,1,2758
1,1,1270,5,2000-12-31 22:00:55,Back to the Future (1985),Comedy|Sci-Fi,F,1,10,48067,1,1069
2,1,1721,4,2000-12-31 22:00:55,Titanic (1997),Drama|Romance,F,1,10,48067,1,1445
3,1,1022,5,2000-12-31 22:00:55,Cinderella (1950),Animation|Children's|Musical,F,1,10,48067,1,862
4,1,2340,3,2000-12-31 22:01:43,Meet Joe Black (1998),Romance,F,1,10,48067,1,1979


In [14]:
if (data['EncodedMovieID'] == 0).any():
    print("The encoder assigned 0 as a value for some EncodedMovieID entries.")
else:
    print("No EncodedMovieID entry has been assigned the value 0.")

if (data['EncodedUserID'] == 0).any():
    print("The encoder assigned 0 as a value for some EncodedUserID entries.")
else:
    print("No EncodedUserID entry has been assigned the value 0.")



No EncodedMovieID entry has been assigned the value 0.
No EncodedUserID entry has been assigned the value 0.


In [15]:
user_sequences = data.groupby('EncodedUserID')['EncodedMovieID'].apply(list).values
print(user_sequences[:2])
print(len(user_sequences))

[list([2758, 1069, 1445, 862, 1979, 1520, 2926, 2412, 1008, 995, 634, 242, 768, 562, 2303, 1631, 1698, 2686, 787, 875, 1632, 1688, 135, 868, 927, 763, 1086, 2405, 2370, 1045, 596, 2514, 497, 2695, 2399, 1960, 869, 998, 551, 2035, 1302, 494, 552, 2298, 649, 546, 1, 1993, 1937, 674, 1315, 1579, 47])
 list([999, 1011, 1018, 2326, 1092, 2539, 1026, 995, 296, 2617, 2459, 1014, 1615, 1008, 550, 2676, 2981, 1548, 482, 920, 2134, 2622, 105, 1735, 2723, 1046, 2686, 1148, 997, 1627, 1623, 769, 1518, 914, 1632, 2984, 3140, 3218, 1058, 1624, 1486, 2337, 1638, 932, 751, 2967, 3078, 2453, 2868, 2652, 247, 1952, 548, 1052, 2655, 1043, 3183, 1625, 1044, 1885, 3163, 827, 1860, 1914, 1221, 2807, 588, 217, 926, 945, 465, 1045, 3360, 1295, 990, 2033, 1996, 1960, 333, 2689, 1064, 3286, 547, 1698, 2197, 424, 2512, 1353, 447, 148, 357, 2934, 2808, 1192, 21, 326, 1288, 1991, 1676, 1921, 1158, 589, 2064, 1490, 1160, 1308, 2123, 1173, 671, 2481, 2688, 345, 1497, 150, 426, 409, 1341, 2242, 1426, 2809, 644, 1672,

In [16]:
def generate_sequences(sequence, window_size=10):
    if len(sequence) < window_size:
        sequence = [0] * (window_size - len(sequence)) + sequence
    return [sequence[i:i + window_size] for i in range(len(sequence) - window_size + 1)]


In [18]:
user_interaction_counts = [len(seq) for seq in user_sequences]
filtered_user_sequences = [
    seq for seq, count in zip(user_sequences, user_interaction_counts)
    if count <= 100  
]
print(len(filtered_user_sequences))

all_sequences = []
for seq in filtered_user_sequences:
    all_sequences.extend(generate_sequences(seq))
print(len(all_sequences))

3132
128306


In [19]:
print(all_sequences[:5])

[[2758, 1069, 1445, 862, 1979, 1520, 2926, 2412, 1008, 995], [1069, 1445, 862, 1979, 1520, 2926, 2412, 1008, 995, 634], [1445, 862, 1979, 1520, 2926, 2412, 1008, 995, 634, 242], [862, 1979, 1520, 2926, 2412, 1008, 995, 634, 242, 768], [1979, 1520, 2926, 2412, 1008, 995, 634, 242, 768, 562]]


In [20]:
all_sequences = torch.tensor(all_sequences, dtype=torch.long)
inputs = all_sequences[:, :-1]  
targets = all_sequences[:, -1]  
print(inputs[:20])

tensor([[2758, 1069, 1445,  862, 1979, 1520, 2926, 2412, 1008],
        [1069, 1445,  862, 1979, 1520, 2926, 2412, 1008,  995],
        [1445,  862, 1979, 1520, 2926, 2412, 1008,  995,  634],
        [ 862, 1979, 1520, 2926, 2412, 1008,  995,  634,  242],
        [1979, 1520, 2926, 2412, 1008,  995,  634,  242,  768],
        [1520, 2926, 2412, 1008,  995,  634,  242,  768,  562],
        [2926, 2412, 1008,  995,  634,  242,  768,  562, 2303],
        [2412, 1008,  995,  634,  242,  768,  562, 2303, 1631],
        [1008,  995,  634,  242,  768,  562, 2303, 1631, 1698],
        [ 995,  634,  242,  768,  562, 2303, 1631, 1698, 2686],
        [ 634,  242,  768,  562, 2303, 1631, 1698, 2686,  787],
        [ 242,  768,  562, 2303, 1631, 1698, 2686,  787,  875],
        [ 768,  562, 2303, 1631, 1698, 2686,  787,  875, 1632],
        [ 562, 2303, 1631, 1698, 2686,  787,  875, 1632, 1688],
        [2303, 1631, 1698, 2686,  787,  875, 1632, 1688,  135],
        [1631, 1698, 2686,  787,  875, 1

In [21]:
print(targets)


tensor([ 995,  634,  242,  ..., 1202, 2310,  985])


In [22]:
print(f'Inputs shape: {inputs.shape}, Targets shape: {targets.shape}')


Inputs shape: torch.Size([128306, 9]), Targets shape: torch.Size([128306])


In [34]:
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split


dataset = TensorDataset(inputs, targets)

total_size = len(dataset)
train_size = int(0.7 * total_size)  
val_size = int(0.15 * total_size)   
test_size = total_size - train_size - val_size  

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

print(f"Train size: {len(train_loader.dataset)}")
print(f"Validation size: {len(val_loader.dataset)}")
print(f"Test size: {len(test_loader.dataset)}")

Train size: 89814
Validation size: 19245
Test size: 19247


In [24]:
for batch_inputs, batch_targets in train_loader:
    print(f'Batch Inputs: {batch_inputs.shape}, Batch Targets: {batch_targets.shape}')
    break

Batch Inputs: torch.Size([256, 9]), Batch Targets: torch.Size([256])


In [25]:
import pickle
with open("movie_mappings.pkl", "wb") as f:
    pickle.dump(
        {
            "encoded_movie_id_to_name": encoded_movie_id_to_name,
            "encoded_movie_name_to_id": encoded_movie_name_to_id,
        },
        f,
    )

In [26]:
num_items = data['EncodedMovieID'].nunique() + 1
print(num_items)

3417


In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads, dropout=0.1):
        super(MultiHeadSelfAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = embedding_dim // num_heads

        assert embedding_dim % num_heads == 0, "Embedding dimension must be divisible by number of heads"

        self.query = nn.Linear(embedding_dim, embedding_dim)
        self.key = nn.Linear(embedding_dim, embedding_dim)
        self.value = nn.Linear(embedding_dim, embedding_dim)

        self.out = nn.Linear(embedding_dim, embedding_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, N, E = x.size()  

        Q = self.query(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        K = self.key(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)
        V = self.value(x).view(B, N, self.num_heads, self.head_dim).transpose(1, 2)

        scores = torch.matmul(Q, K.transpose(-2, -1)) / (self.head_dim ** 0.5)
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        output = torch.matmul(attn, V)
        output = output.transpose(1, 2).contiguous().view(B, N, E)

        return self.out(output)

class PointWiseFeedForward(nn.Module):
    def __init__(self, embedding_dim, ff_dim, dropout=0.1):
        super(PointWiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(embedding_dim, ff_dim)
        self.dropout = nn.Dropout(dropout)
        self.linear2 = nn.Linear(ff_dim, embedding_dim)

    def forward(self, x):
        x = self.dropout(F.relu(self.linear1(x)))
        return self.linear2(x)

class CustomTransformerEncoderLayer(nn.Module):
    def __init__(self, embedding_dim, num_heads, ff_dim, dropout=0.1):
        super(CustomTransformerEncoderLayer, self).__init__()
        self.msa = MultiHeadSelfAttention(embedding_dim, num_heads, dropout)
        self.ffn = PointWiseFeedForward(embedding_dim, ff_dim, dropout)

        self.norm1 = nn.LayerNorm(embedding_dim)
        self.norm2 = nn.LayerNorm(embedding_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x + self.dropout(self.msa(self.norm1(x)))
        x = x + self.dropout(self.ffn(self.norm2(x)))
        return x

class ModelAugmentation(nn.Module):
    def __init__(self, embedding_dim, dropout=0.1):
        super(ModelAugmentation, self).__init__()
        self.ffn = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(embedding_dim, embedding_dim)
        )
        self.norm = nn.LayerNorm(embedding_dim)

    def forward(self, x):
        return self.norm(self.ffn(x) + x)

class UET4Rec(nn.Module):
    def __init__(self, num_items, embedding_dim=64, nhead=4, max_len=50, dropout=0.2, ff_dim=256):
        super(UET4Rec, self).__init__()

        self.item_embedding = nn.Embedding(num_items, embedding_dim, padding_idx=0)

        self.position_embedding = nn.Embedding(max_len, embedding_dim)

        self.encoder1 = nn.Sequential(
            nn.Conv1d(embedding_dim, embedding_dim // 2, kernel_size=5, padding=2),
            nn.BatchNorm1d(embedding_dim // 2),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout)
        )

        self.encoder2 = nn.Sequential(
            nn.Conv1d(embedding_dim // 2, embedding_dim // 4, kernel_size=5, padding=2),
            nn.BatchNorm1d(embedding_dim // 4),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout)
        )

        self.encoder3 = nn.Sequential(
            nn.Conv1d(embedding_dim // 4, embedding_dim // 8, kernel_size=5, padding=2)
        )

        self.transformer = nn.ModuleList([
            CustomTransformerEncoderLayer(embedding_dim // 8, nhead, ff_dim, dropout) for _ in range(3)
        ])

        self.decoder1 = nn.Sequential(
            nn.Conv1d(embedding_dim // 8, embedding_dim // 4, kernel_size=5, padding=2),
            nn.BatchNorm1d(embedding_dim // 4),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout)
        )

        self.decoder2 = nn.Sequential(
            nn.Conv1d(embedding_dim // 4, embedding_dim // 2, kernel_size=5, padding=2),
            nn.BatchNorm1d(embedding_dim // 2),
            nn.LeakyReLU(0.1),
            nn.Dropout(dropout)
        )

        self.decoder3 = nn.Sequential(
            nn.Conv1d(embedding_dim // 2, embedding_dim, kernel_size=5, padding=2)
        )

        self.ma = ModelAugmentation(embedding_dim, dropout)

        self.fc = nn.Linear(embedding_dim, num_items)

    def forward(self, x):
        batch_size, seq_len = x.size()

        positions = torch.arange(0, seq_len).unsqueeze(0).repeat(batch_size, 1).to(x.device)
        x = self.item_embedding(x) + self.position_embedding(positions)

        x = x.permute(0, 2, 1)  
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(enc1)
        enc3 = self.encoder3(enc2)

        x = enc3.permute(0, 2, 1)  
        for layer in self.transformer:
            x = layer(x)
        x = x.permute(0, 2, 1)  

        dec1 = self.decoder1(x + enc3)  
        dec2 = self.decoder2(dec1 + enc2)  
        dec3 = self.decoder3(dec2 + enc1)  

        x = dec3[:, :, -1]  
        x = self.ma(x)  
        out = self.fc(x)  

        return out


In [28]:
embedding_dim = 128
nhead = 4
ff_dim = 256
dropout = 0.2
max_len = 9

model = UET4Rec(
    num_items=num_items,
    embedding_dim=embedding_dim,
    nhead=nhead,
    max_len=max_len,
    dropout=dropout,
    ff_dim=ff_dim
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(model)

UET4Rec(
  (item_embedding): Embedding(3417, 128, padding_idx=0)
  (position_embedding): Embedding(9, 128)
  (encoder1): Sequential(
    (0): Conv1d(128, 64, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Dropout(p=0.2, inplace=False)
  )
  (encoder2): Sequential(
    (0): Conv1d(64, 32, kernel_size=(5,), stride=(1,), padding=(2,))
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.1)
    (3): Dropout(p=0.2, inplace=False)
  )
  (encoder3): Sequential(
    (0): Conv1d(32, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  )
  (transformer): ModuleList(
    (0-2): 3 x CustomTransformerEncoderLayer(
      (msa): MultiHeadSelfAttention(
        (query): Linear(in_features=16, out_features=16, bias=True)
        (key): Linear(in_features=16, out_features=16, bias=True)
        (value)

In [29]:
import torch
import torch.optim as optim
from tqdm import tqdm

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1000
save_interval = 50

print("Starting Training...")

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with tqdm(train_loader, desc=f"Epoch [{epoch}/{num_epochs}]", unit="batch") as progress_bar:
        for batch_inputs, batch_targets in progress_bar:
            batch_inputs, batch_targets = batch_inputs.to(device), batch_targets.to(device)

            optimizer.zero_grad()

            outputs = model(batch_inputs)  
            loss = criterion(outputs, batch_targets)  

            loss.backward()  
            optimizer.step()  

            total_loss += loss.item()

            _, predicted = torch.max(outputs, dim=1)
            correct_predictions += (predicted == batch_targets).sum().item()
            total_predictions += batch_targets.size(0)

            progress_bar.set_postfix({"Batch Loss": loss.item()})

    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions

    print(f"Epoch [{epoch}/{num_epochs}], Training Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    if epoch % save_interval == 0:
        model_save_path = f"uet4rec64_ml_model_epoch_{epoch}.pth"
        torch.save(model.state_dict(), model_save_path)
        print(f"Model saved at '{model_save_path}'")

final_model_path = "uet4rec64_ml_model_final.pth"
torch.save(model.state_dict(), final_model_path)
print(f"Final model saved as '{final_model_path}'")

print("Training Completed.")


Starting Training...


Epoch [1/1000]: 100%|████████████████████████████████████████████| 351/351 [00:34<00:00, 10.29batch/s, Batch Loss=7.13]


Epoch [1/1000], Training Loss: 7.2097, Accuracy: 0.0062


Epoch [2/1000]: 100%|████████████████████████████████████████████| 351/351 [00:25<00:00, 13.84batch/s, Batch Loss=6.32]


Epoch [2/1000], Training Loss: 6.5303, Accuracy: 0.0224


...

Epoch [998/1000], Training Loss: 1.0253, Accuracy: 0.7090


Epoch [999/1000]: 100%|██████████████████████████████████████████| 351/351 [00:24<00:00, 14.16batch/s, Batch Loss=1.06]


Epoch [999/1000], Training Loss: 1.0262, Accuracy: 0.7104


Epoch [1000/1000]: 100%|████████████████████████████████████████| 351/351 [00:24<00:00, 14.56batch/s, Batch Loss=0.929]

Epoch [1000/1000], Training Loss: 1.0182, Accuracy: 0.7109
Model saved at 'uet4rec64_ml_model_epoch_1000.pth'
Final model saved as 'uet4rec64_ml_model_final.pth'
Training Completed.


In [30]:
import numpy as np
import torch
from tqdm import tqdm
from sklearn.metrics import ndcg_score


model = UET4Rec(
    num_items=num_items,
    embedding_dim=embedding_dim,
    nhead=nhead,
    max_len=max_len,
    dropout=dropout,
    ff_dim=ff_dim
)



In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("uet4rec64_ml_model_final.pth", map_location=device)) 
model.to(device)
model.eval() 
print("Model loaded successfully.")

Model loaded successfully.


C:\Users\npavi\AppData\Local\Temp\ipykernel_13796\252579999.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("uet4rec64_ml_model_final.pt

In [35]:



def evaluate_hit_ndcg_multi_k(model, test_loader, ks=[5, 10, 20]):
    model.eval()
    total_hr = {k: 0 for k in ks}
    total_ndcg = {k: 0 for k in ks}
    total_samples = 0

    with torch.no_grad():
        for batch_inputs, batch_targets in tqdm(test_loader, desc="Evaluating HR & NDCG for multiple k"):
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            outputs = model(batch_inputs)  
            _, top_k_indices = torch.topk(outputs, k=max(ks), dim=1) 

            for i in range(batch_targets.size(0)):
                target = batch_targets[i].item()
                predictions = top_k_indices[i].cpu().numpy()

                for k in ks:
                    top_k_preds = predictions[:k]

                    if target in top_k_preds:
                        total_hr[k] += 1

                    if target in top_k_preds:
                        rank = np.where(top_k_preds == target)[0][0] + 1
                        total_ndcg[k] += 1 / np.log2(rank + 1)

            total_samples += batch_targets.size(0)

    hr = {k: total_hr[k] / total_samples for k in ks}
    ndcg = {k: total_ndcg[k] / total_samples for k in ks}

    return hr, ndcg

k_values = [5, 10, 20]
hr, ndcg = evaluate_hit_ndcg_multi_k(model, test_loader, ks=k_values)

print("\nMetrics Summary:")
for k in k_values:
    print(f"HR@{k}: {hr[k]:.4f}")
    print(f"NDCG@{k}: {ndcg[k]:.4f}")


Evaluating HR & NDCG for multiple k: 100%|█████████████████████████████████████████████| 76/76 [00:08<00:00,  9.04it/s]


Metrics Summary:
HR@5: 0.7277
NDCG@5: 0.6979
HR@10: 0.7540
NDCG@10: 0.7064
HR@20: 0.7824
NDCG@20: 0.7135
